In [2]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
cap.set(100,700)
cap.set(100,350)
def reorder(myPoints):

    myPoints = myPoints.reshape((4, 2))
    myPointsNew = np.zeros((4, 1, 2), dtype=np.int32)
    add = myPoints.sum(1)

    myPointsNew[0] = myPoints[np.argmin(add)]
    myPointsNew[3] =myPoints[np.argmax(add)]
    diff = np.diff(myPoints, axis=1)
    myPointsNew[1] =myPoints[np.argmin(diff)]
    myPointsNew[2] = myPoints[np.argmax(diff)]

    return myPointsNew


def biggestContour(contours):
    biggest = np.array([])
    max_area = 0
    for i in contours:
        area = cv2.contourArea(i)
        if area > 5000:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.02 * peri, True)
            if area > max_area and len(approx) == 4:
                biggest = approx
                max_area = area
    return biggest,max_area

def drawRectangle(frame,biggest,thickness):
    cv2.line(frame, (biggest[0][0][0], biggest[0][0][1]), (biggest[1][0][0], biggest[1][0][1]), (255, 0, 0), thickness)
    cv2.line(frame, (biggest[0][0][0], biggest[0][0][1]), (biggest[2][0][0], biggest[2][0][1]), (255, 0, 0), thickness)
    cv2.line(frame, (biggest[3][0][0], biggest[3][0][1]), (biggest[2][0][0], biggest[2][0][1]), (255, 0, 0), thickness)
    cv2.line(frame, (biggest[3][0][0], biggest[3][0][1]), (biggest[1][0][0], biggest[1][0][1]), (255, 0, 0), thickness)

    return frame


def empty(a):
    pass

cv2.namedWindow("Set")
cv2.resizeWindow("Set", 600,80)
cv2.createTrackbar("threshold1","Set",70,255,empty)
cv2.createTrackbar("threshold2","Set",200,255,empty)

# def getCountours(frame,imgCountour):
#     countours, hierarchy = cv2.findContours(frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
#     cv2.drawContours(imgCountour,countours, -1, (0,255,0),5)
    
count=0
count2=0
while True:
    _, frame = cap.read()
    imgCountours=frame.copy()
    imgBigContour = frame.copy()
    
    blurred_frame = cv2.GaussianBlur(frame, (7, 7), 1)
    gray = cv2.cvtColor(blurred_frame, cv2.COLOR_BGR2GRAY)
    
    threshold1 = cv2.getTrackbarPos("threshold1", "Set")
    threshold2 = cv2.getTrackbarPos("threshold2", "Set")
    
    edged= cv2.Canny(gray, threshold1, threshold2)
#     hsv = cv2.cvtColor(blurred_frame, cv2.COLOR_BGR2HSV)
    
#     lower_blue = np.array([38, 255, 0])
#     upper_blue = np.array([129, 255, 0])
#     mask = cv2.inRange(hsv, lower_blue, upper_blue)    
#     _, contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
#     cv2.drawContours(frame, contours, -1, (0, 255, 0), 3)        
    
#     cv2.imshow("Mask", mask)
    kernel=np.ones((5,5))
    frameDil=cv2.dilate(edged,kernel, iterations=1)
    contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(imgCountours,contours, -1, (0,255,0),10)
    
    biggest, maxArea = biggestContour(contours) # FIND THE BIGGEST CONTOUR
    if biggest.size != 0:
        biggest= reorder(biggest)
        cv2.drawContours(imgBigContour, biggest, -1, (255, 0,  0), 20) # DRAW THE BIGGEST CONTOUR
        imgBigContour = drawRectangle(imgBigContour,biggest,2)
        pts1 = np.float32(biggest) # PREPARE POINTS FOR WARP
        pts2 = np.float32([[0, 0],[500, 0], [0, 700],[500, 700]]) # PREPARE POINTS FOR WARP
        matrix = cv2.getPerspectiveTransform(pts1, pts2) 
        imgWarpPerspective = cv2.warpPerspective(frame, matrix, (500, 700))

        #REMOVE 20 PIXELS FORM EACH SIDE
        imgWarpPerspective=imgWarpPerspective[20:imgWarpPerspective.shape[0] - 20, 20:imgWarpPerspective.shape[1] - 20]
        imgWarpPerspective = cv2.resize(imgWarpPerspective,(500,700))

    
#     cv2.imshow("Frame",frame)    
#     cv2.imshow("Dil",frameDil)
#Tahap 1 Scan Dokumen Dulu Baru munculin imgWarpColored
    cv2.imshow("Countour",imgCountours)
    cv2.imshow("SecondContours",imgBigContour)    
    cv2.imshow("Warp",imgWarpPerspective)    

    if cv2.waitKey(1) & 0xFF == ord('s'):

        cv2.imwrite("Documents\Contours"+str(count)+".jpg",imgCountours)
        count += 1
        cv2.putText(imgCountours, "Tersimpan", (int(imgCountours.shape[1] / 2) - 100, int(imgCountours.shape[0] / 2)),
                    cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255), 5, cv2.LINE_AA)
        cv2.imshow("Countour",imgCountours)
        cv2.waitKey(3800)
    
    if cv2.waitKey(1) & 0xFF == ord('w'):

        cv2.imwrite("Documents\Scan"+str(count)+".jpg",imgWarpPerspective)
        count2 += 1
        cv2.putText(imgWarpPerspective, "Simpan", (int(imgWarpPerspective.shape[1] / 2) - 100, int(imgWarpPerspective.shape[0] / 2)),
                    cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255), 5, cv2.LINE_AA)
        cv2.imshow("Warp",imgWarpPerspective)    
        cv2.waitKey(3800)
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
    
       
cap.release()
cv2.destroyAllWindows()